In [1]:
from keras_preprocessing.sequence import pad_sequences
import os
import fnmatch
import cv2
import numpy as np
import string
import time
import sys

In [2]:
# char_list:   'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789'
# total number of our output classes: len(char_list)
char_list = string.ascii_letters+string.digits
 
def encode_to_labels(txt):
    # encoding each output word into digits
    dig_lst = []
    for index, char in enumerate(txt):
        try:
            dig_lst.append(char_list.index(char))
        except:
            print(char)
        
    return dig_lst

In [3]:

path = r'C:\Aditya\Project Book\New\mnt\ramdisk\max\90kDICT32px'
 
# lists for training dataset
training_img = np.zeros((390000, 32, 128, 1),dtype='uint8')
training_txt = []
train_input_length = np.zeros((390000, ),dtype='uint8')
train_label_length = np.zeros((390000, ),dtype='uint8')
 
max_label_len = 0
 
i =0
flag = 0

for root, dirnames, filenames in os.walk(path):
  for f_name in fnmatch.filter(filenames, '*.jpg'):
      # read input image and convert into gray scale image
      try:
        img = cv2.cvtColor(cv2.imread(os.path.join(root, f_name)), cv2.COLOR_BGR2GRAY)   

        # convert each image of shape (32, 128, 1)
        w, h = img.shape
        if h > 128 or w > 32:
            continue

        if w < 32:
            add_zeros = np.ones((32-w, h))*255
            img = np.concatenate((img, add_zeros))

        if h < 128:
            add_zeros = np.ones((32, 128-h))*255
            img = np.concatenate((img, add_zeros), axis=1)
        img = np.expand_dims(img , axis = 2)
        
        # get the text from the image
        txt = f_name.split('_')[1]
        
        # compute maximum length of the text
        if len(txt) > max_label_len:
            # print()
            max_label_len = len(txt)

        if len(txt) == 0:
          print(txt)
          continue 
            
        # orig_txt.append(txt)   
        train_label_length[i]= len(txt)
        train_input_length[i]= 31
        training_img[i]= img
        training_txt.append(encode_to_labels(txt)) 

        sys.stdout.write('\r' + str(i) + ' ' + str(np.count_nonzero(train_label_length)))           
        
        # break the loop if total data is 390000
        if i == 389999:
            flag = 1
            break
        i+=1

      except:
        print('\n' + os.path.join(root, f_name))
      
  if flag == 1:
      break
    
# pad each output label to maximum text length
train_padded_txt = pad_sequences(training_txt, maxlen=max_label_len, padding='post', value = len(char_list))

389999 390000

In [4]:
train_label_length = np.array(train_label_length, dtype='uint8')
train_input_length = np.array(train_input_length, dtype='uint8')
training_img = np.array(training_img, dtype='uint8')
train_padded_txt = np.array(train_padded_txt,dtype='uint8')

In [5]:
print(train_label_length.shape)
print(train_input_length.shape)
print(training_img.shape)
print(train_padded_txt.shape)

(390000,)
(390000,)
(390000, 32, 128, 1)
(390000, 22)


In [6]:
np.save(r'C:\Aditya\Project Book\New\numpy_save\train_label_length.npy',train_label_length)
np.save(r'C:\Aditya\Project Book\New\numpy_save\train_input_length.npy',train_input_length)
np.save(r'C:\Aditya\Project Book\New\numpy_save\training_img.npy',training_img)
np.save(r'C:\Aditya\Project Book\New\numpy_save\train_padded_txt.npy',train_padded_txt)
np.save(r'C:\Aditya\Project Book\New\numpy_save\max_label_len.npy',max_label_len)